<a href="https://colab.research.google.com/github/anthropics/anthropic-cookbook/blob/main/misc/pdf_upload_summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "Uploading" PDFs to Claude Via the API

One really nice feature of [Claude.ai](https://www.claude.ai) is the ability to upload PDFs. Let's mock up that feature in a notebook, and then test it out by summarizing a long PDF.

In [11]:
!curl -O https://arxiv.org/pdf/2212.08073.pdf

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2039k  100 2039k    0     0  11.8M      0 --:--:-- --:--:-- --:--:-- 11.8M


Now, we'll use the pypdf package to read the pdf. It's not identical to what Claude.ai uses behind the scenes, but it's pretty close. Note that this type of extraction only works for text content within PDFs. If your PDF contains visual elements (like charts and graphs) refer to the cookbook recipes in our [Multimodal folder](
https://github.com/anthropics/anthropic-cookbook/tree/main/multimodal) for techniques.

In [ ]:
%pip install pypdf

In [12]:
from pypdf import PdfReader

reader = PdfReader("2212.08073.pdf")
number_of_pages = len(reader.pages)
text = ''.join(page.extract_text() for page in reader.pages)
print(text[:2155])

Constitutional AI: Harmlessness from AI Feedback
Yuntao Bai∗, Saurav Kadavath, Sandipan Kundu, Amanda Askell, Jackson Kernion,
Andy Jones, Anna Chen, Anna Goldie, Azalia Mirhoseini, Cameron McKinnon,
Carol Chen, Catherine Olsson, Christopher Olah, Danny Hernandez, Dawn Drain,
Deep Ganguli, Dustin Li, Eli Tran-Johnson, Ethan Perez, Jamie Kerr, Jared Mueller,
Jeffrey Ladish, Joshua Landau, Kamal Ndousse, Kamile Lukosuite, Liane Lovitt,
Michael Sellitto, Nelson Elhage, Nicholas Schiefer, Noemi Mercado, Nova DasSarma,
Robert Lasenby, Robin Larson, Sam Ringer, Scott Johnston, Shauna Kravec,
Sheer El Showk, Stanislav Fort, Tamera Lanham, Timothy Telleen-Lawton, Tom Conerly,
Tom Henighan, Tristan Hume, Samuel R. Bowman, Zac Hatﬁeld-Dodds, Ben Mann,
Dario Amodei, Nicholas Joseph, Sam McCandlish, Tom Brown, Jared Kaplan∗
Anthropic
Abstract
As AI systems become more capable, we would like to enlist their help to supervise
other AIs. We experiment with methods for training a harmless AI assistant

With the paper downloaded and in memory, we can ask Claude to perform various fun tasks with it.

In [3]:
from anthropic import Anthropic
client = Anthropic()
MODEL_NAME = "claude-3-opus-20240229"

In [4]:
def get_completion(client, prompt):
    return client.messages.create(
        model=MODEL_NAME,
        max_tokens=2048,
        messages=[{
            "role": 'user', "content":  prompt
        }]
    ).content[0].text

In [14]:
completion = get_completion(client,
    f"""Here is an academic paper: <paper>{text}</paper>

Please do the following:
1. Summarize the abstract at a kindergarten reading level. (In <kindergarten_abstract> tags.)
2. Write the Methods section as a recipe from the Moosewood Cookbook. (In <moosewood_methods> tags.)
3. Compose a short poem epistolizing the results in the style of Homer. (In <homer_results> tags.)
"""
)
print(completion)

Here is my attempt at the requested tasks:

<kindergarten_abstract>
This paper talks about making computer helpers that are nice and don't do anything bad. The helpers learn to be good by reading a list of rules and checking their own work to make sure they follow the rules. Then the helpers get even better at being nice by playing a game where they give advice and score points for saying things that help people and don't hurt anyone.
</kindergarten_abstract>

<moosewood_methods>
Constitutional AI Casserole

Ingredients:
- 1 large language model, pretrained
- 16 cups of constitutional principles
- 182,831 red teaming prompts
- 135,296 helpfulness prompts
- A dash of chain-of-thought reasoning

Instructions:
1. Preheat your neural networks to a learning rate of 0.5.
2. In a large mixing bowl, combine the pretrained language model with the constitutional principles. Stir until the model is thoroughly coated in ethics.
3. Pour in the red teaming prompts and helpfulness prompts. Mix well u